# Tarefa 4

Alunos:

* Andreza - RA: 164213
* Gil - RA: 225323
* Yan - RA: 118982

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("dados4.csv")
X = df.drop(['V15'], axis=1)
y = df.filter(['V15'], axis=1)

## Preprocessamento

In [3]:
X = pd.get_dummies(df, columns=['V5', 'V6', 'V12'])

In [4]:
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)

## Métodos

In [5]:
methods = {
    'Logistic_Regression': LogisticRegression(penalty='none'),
    'Logistic_Regression_l2': LogisticRegression(penalty='l2'),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis(),
    'SVM_Linear': LinearSVC(),
    'SVM_RBF': SVC(kernel='rbf'),
    'Naive_Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'Decision_Tree': DecisionTreeClassifier(),
    'Random_Forest': RandomForestClassifier(),
    'GBM': GradientBoostingClassifier(),
}

params={
    'Logistic_Regression': {},
    'Logistic_Regression_l2': {'C': 10**(np.random.uniform(-3, 3, 10))},
    'LDA': {},
    'QDA': {},
    'SVM_Linear': {'C': 2**(np.random.uniform(-5, 15, 10))},
    'SVM_RBF': {'C': 2**(np.random.uniform(-5, 15, 10)),
                'gamma': 2**(np.random.uniform(-9, 3, 10))},
    'Naive_Bayes': {},
    'KNN': {'n_neighbors': np.random.randint(1, 301, 10)},
    'MLP': {'hidden_layer_sizes': np.arange(5, 21, 3)},
    'Decision_Tree': {'ccp_alpha': np.random.uniform(0.04, 0.0, 10)},
    'Random_Forest': {'n_estimators': [10, 100, 1000],
                      'max_features': [5, 8, 10]},
    'GBM': {'n_estimators':np.random.randint(5, 100, 10),
            'learning_rate': np.random.uniform(0.01, 0.3, 10),
            'max_depth': [2, 3]}
}

## Iterações

In [6]:
results = {
    'clf': [],
    'mean_auc_outer': []#,
#    'best_params': []
}

for method in methods:
    print(method)
    auc_outer = []
    for iteration in range(4):
        print(iteration)
        X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y), test_size=0.3)
        clf = GridSearchCV(
                    methods[method],
                    params[method],
                    cv=StratifiedKFold(n_splits=3),
                    scoring='roc_auc',
                    refit=True)
        clf.fit(X_train,y_train)
        # get prediction on test (outer loop)
        y_pred = clf.predict(X_test)
        # calculate auc and save
        auc = roc_auc_score(y_test, y_pred)
        auc_outer.append(auc)
#        break
    mean_auc = np.mean(auc_outer)
    results['clf'].append(method)
    results['mean_auc_outer'].append(mean_auc)
#    results['best_params'].append(clf.best_params_)
#    break


Logistic_Regression
0
1
2
3
Logistic_Regression_l2
0
1
2
3
LDA
0
1
2
3
QDA
0
1
2
3
SVM_Linear
0
1
2
3
SVM_RBF
0
1
2
3
Naive_Bayes
0
1
2
3
KNN
0
1
2
3
MLP
0
1
2
3
Decision_Tree
0
1
2
3
Random_Forest
0
1
2
3
GBM
0
1
2
3


In [7]:
results_df = pd.DataFrame(results)
results_df

,clf,mean_auc_outer
0,Logistic_Regression,1.000000
1,Logistic_Regression_l2,1.000000
2,LDA,1.000000
3,QDA,0.993962
4,SVM_Linear,1.000000
5,SVM_RBF,1.000000
6,Naive_Bayes,0.994253
7,KNN,0.907377
8,MLP,0.971907
9,Decision_Tree,1.000000
